In [ ]:
# Importações e instalação das dependências
!pip install pydicom nibabel schedule

import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import pydicom
import nibabel as nib
import numpy as np
import schedule
import time

class DicomToNiftiConverter:
    def __init__(self, input_folder, output_folder, email_config):
        """
        Inicializa o conversor de DICOM para NIfTI.

        :param input_folder: Pasta onde estão os arquivos DICOM.
        :param output_folder: Pasta onde os arquivos NIfTI serão salvos.
        :param email_config: Configurações de e-mail (dicionário com 'smtp_server', 'port', 'sender_email', 'receiver_email', 'password').
        """
        self.input_folder = input_folder
        self.output_folder = output_folder
        self.email_config = email_config
        os.makedirs(self.output_folder, exist_ok=True)

    def convert_dicom_to_nifti(self, dicom_file, output_file):
        """
        Converte um arquivo DICOM para NIfTI.

        :param dicom_file: Caminho do arquivo DICOM.
        :param output_file: Caminho onde o arquivo NIfTI será salvo.
        """
        try:
            # Ler o arquivo DICOM
            dicom_data = pydicom.dcmread(dicom_file)
            pixel_array = dicom_data.pixel_array

            # Criar um objeto NIfTI
            nifti_image = nib.Nifti1Image(pixel_array, affine=np.eye(4))

            # Salvar o arquivo NIfTI
            nib.save(nifti_image, output_file)
            print(f"Arquivo convertido: {output_file}")
        except Exception as e:
            print(f"Erro ao converter {dicom_file}: {e}")

    def find_unconverted_files(self):
        """
        Busca arquivos DICOM que ainda não foram convertidos.

        :return: Lista de caminhos de arquivos DICOM não convertidos.
        """
        unconverted_files = []
        for root, _, files in os.walk(self.input_folder):
            for file in files:
                if file.endswith(".dcm"):
                    dicom_path = os.path.join(root, file)
                    nifti_path = os.path.join(self.output_folder, file.replace(".dcm", ".nii"))
                    if not os.path.exists(nifti_path):
                        unconverted_files.append(dicom_path)
        return unconverted_files

    def send_email_with_attachment(self, file_path):
        """
        Envia um e-mail com o arquivo NIfTI anexado.

        :param file_path: Caminho do arquivo NIfTI a ser enviado.
        """
        try:
            msg = MIMEMultipart()
            msg['From'] = self.email_config['sender_email']
            msg['To'] = self.email_config['receiver_email']
            msg['Subject'] = "Arquivo NIfTI Convertido"

            # Anexar o arquivo
            with open(file_path, "rb") as attachment:
                part = MIMEBase("application", "octet-stream")
                part.set_payload(attachment.read())
                encoders.encode_base64(part)
                part.add_header(
                    "Content-Disposition",
                    f"attachment; filename={os.path.basename(file_path)}"
                )
                msg.attach(part)

            # Conectar ao servidor SMTP e enviar o e-mail
            with smtplib.SMTP(self.email_config['smtp_server'], self.email_config['port']) as server:
                server.starttls()
                server.login(self.email_config['sender_email'], self.email_config['password'])
                server.sendmail(self.email_config['sender_email'], self.email_config['receiver_email'], msg.as_string())
            print(f"E-mail enviado com sucesso: {file_path}")
        except Exception as e:
            print(f"Erro ao enviar e-mail: {e}")

    def process_files(self):
        """
        Processa todos os arquivos DICOM não convertidos, converte-os para NIfTI e envia por e-mail.
        """
        unconverted_files = self.find_unconverted_files()
        for dicom_file in unconverted_files:
            nifti_file = os.path.join(self.output_folder, os.path.basename(dicom_file).replace(".dcm", ".nii"))
            self.convert_dicom_to_nifti(dicom_file, nifti_file)
            self.send_email_with_attachment(nifti_file)

    def schedule_daily_task(self, times):
        """
        Agenda a execução da tarefa diariamente nos horários especificados.

        :param times: Lista de strings no formato "HH:MM" (24h) indicando os horários de execução.
        """
        for time_str in times:
            schedule.every().day.at(time_str).do(self.process_files)
            print(f"Tarefa agendada para executar diariamente às {time_str}.")

        while True:
            schedule.run_pending()
            time.sleep(1)




In [ ]:
# Configurações
input_folder = "/content/dicom_files"  # Pasta com arquivos DICOM
output_folder = "/content/nifti_files"  # Pasta para salvar arquivos NIfTI
email_config = {
    "smtp_server": "smtp.gmail.com",
    "port": 587,
    "sender_email": "seu_email@gmail.com",
    "receiver_email": "destinatario@gmail.com",
    "password": "sua_senha"
}

# Criar e iniciar o conversor
converter = DicomToNiftiConverter(input_folder, output_folder, email_config)

# Executar a tarefa imediatamente
converter.process_files()

In [ ]:
# Configurações
input_folder = "/content/dicom_files"  # Pasta com arquivos DICOM
output_folder = "/content/nifti_files"  # Pasta para salvar arquivos NIfTI
email_config = {
    "smtp_server": "smtp.gmail.com",
    "port": 587,
    "sender_email": "seu_email@gmail.com",
    "receiver_email": "destinatario@gmail.com",
    "password": "sua_senha"
}

# Criar e iniciar o conversor
converter = DicomToNiftiConverter(input_folder, output_folder, email_config)

# Horários para execução diária (exemplo: 09:00, 15:00, 21:00)
daily_times = ["09:00", "15:00", "21:00"]

# Agendar a tarefa para rodar diariamente nos horários especificados
converter.schedule_daily_task(daily_times)